In [68]:
import pandas as pd
import seaborn as sns
import re
from nltk.corpus import stopwords
from gensim import similarities

import math
import time
import datetime
import numpy as np

from scipy import spatial
import ast

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

from sklearn.metrics import classification_report
from sklearn.metrics import PrecisionRecallDisplay

import gensim
import nltk
from nltk.tokenize import sent_tokenize
import string
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from openai.embeddings_utils import cosine_similarity

from gensim.models import Word2Vec
from gensim import similarities
from sklearn.metrics.pairwise import cosine_similarity
from openai.embeddings_utils import cosine_similarity, get_embedding

## Embed sentence 

In [69]:
df = pd.read_excel('amazon_review JOIN.xlsx')

In [32]:
# df_full = pd.read_excel('amazon_review_processed_full.xlsx')

In [70]:
df['Review Content new'] = df['Review Content'].apply(lambda x: str(x) if x is not None else '') 

In [71]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

df['Cleaned Reviews'] = df['Review Content new'].apply(clean_text)

In [35]:
# for i in df['Final Reviews'].head():
#     print(i)

In [72]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [73]:
df['Final Reviews'] = df['Cleaned Reviews'].apply(remove_stopwords)

In [74]:
# def split_text(text):
#     sentences = sent_tokenize(str(text))
#     return sentences 

# def remove_punc(text):
#     remove = str.maketrans('','',string.punctuation)
#     remove_punc = [sentence.translate(remove) for sentence in text]
#     return remove_punc

def brand(model):
    match = re.search(r'^(HP|Epson|Canon)', model, re.IGNORECASE)
    if match:
        return match.group(0)

In [75]:
df['Brand'] = df['Review Model'].apply(brand)
# df['Review Content split'] =  df['Review Content'].apply(split_text)

In [76]:
df.head()

,Review Model,Retailer,Comp Model,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,URL,list price,rating count,overall rating,Review Content new,Cleaned Reviews,Final Reviews,Brand
0,Canon Pixma TS6420a,Amazon,NaN,2023-08-10,Ernest Birkholz,5,Works great,🖨 was easy to install and works great.,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B09TG52XT4/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2&filterByStar=five_star,129.99,285,4.2,🖨 was easy to install and works great.,was easy to install and works great,easy install works great,Canon
1,HP OfficeJet Pro 9015e,Amazon,Epson - Workforce 4820,2022-06-04,mattey,3,spunky mid size printer,🔆Slower print speed than what I’m used to (old model..HP-OJ-Pro. 8600) and when it’s (new 9015e) printing it sounds like it’s breaking apart inside. Setup was annoyingly complex luckily I researched the HP+ & watched reviews (not my thing so I opted NOT to enroll - important to know details before enrolling) But the instant ink program - its terrific hope they don’t ruin it🥴 overall it’s a spunky mid size printer good enough for the typical home office🥴☀️🔆,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B08QR6P8KV/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=4&filterByStar=three_star,289.99,285,4.2,🔆Slower print speed than what I’m used to (old model..HP-OJ-Pro. 8600) and when it’s (new 9015e) printing it sounds like it’s breaking apart inside. Setup was annoyingly complex luckily I researched the HP+ & watched reviews (not my thing so I opted NOT to enroll - important to know details before enrolling) But the instant ink program - its terrific hope they don’t ruin it🥴 overall it’s a spunky mid size printer good enough for the typical home office🥴☀️🔆,slower print speed than what im used to old modelhpojpro and when its new e printing it sounds like its breaking apart inside setup was annoyingly complex luckily i researched the hp watched reviews not my thing so i opted not to enroll important to know details before enrolling but the instant ink program its terrific hope they dont ruin it overall its a spunky mid size printer good enough for the typical home office,slower print speed im used old modelhpojpro new e printing sounds like breaking apart inside setup annoyingly complex luckily researched hp watched reviews thing opted enroll important know details enrolling instant ink program terrific hope dont ruin overall spunky mid size printer good enough typical home office,HP
2,Canon PIXMA MG3620,Amazon,NaN,2023-03-15,Maria D,4,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,𝙻𝚘𝚟𝚎 𝚒𝚝,Verified Purchase,NaN,NaN,https://www.amazon.com/Canon-MG3620-Wireless-Printer-Printing/product-reviews/B010A7TZ76/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=10&filterByStar=four_star,79.99,285,4.2,𝙻𝚘𝚟𝚎 𝚒𝚝,,,Canon
3,Epson - ET-3830,Amazon,NaN,2022-11-17,Ryan H,5,Shaq knows what he's talking about,"Yup, this printer is a slam dunk. :)What an upgrade! Paid a little more up front than an equivalent printer with the throw away types of cartridges, but will save hundreds a year with the tank system. No longer miserly with what I print by trying to find the absolute lowest possible ink usage settings and always in b&w. I can print from my macbooks, iphone, ipad easy breezy. They've done a great job engineering the ink filling process and it's completely mess free.",Verified Purchase,12.0,NaN,https://www.amazon.com/product-reviews/B096NBPHZ7/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2&filterByStar=five_star,399.99,285,4.2,"Yup, this printer is a slam dunk. :)What an upgrade! Paid a little more up front than an equivalent printer with the throw away types of cartridges, but will save hundreds a year with the tank system. No longer miserly with what I print by trying to find the absolute lowest possible ink usage se

In [72]:
# import gensim.downloader as api 
# model = api.load("glove-wiki-gigaword-50")

In [106]:
import tensorflow as tf
import tensorflow_hub as hub 
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed(text,model):
    embeddings = model(text)
    return [embedding.numpy() for embedding in embeddings]

# def embed(text, model):
#     embeddings = model(text)
#     return [embedding.numpy().tolist() for embedding in embeddings]

df['Embed_sentence'] = embed(df['Final Reviews'], model)

ValueError: When input_signature is provided, all inputs to the Python function must be convertible to tensors:
  inputs: (
    0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  easy install works great
1                                                                                                                                                                                                                               slower print speed im used old modelhpojpro new e printing sounds like breaking apart inside setup annoyingl

In [89]:
df.to_csv('amazon_review_embedded.csv')

In [83]:
df.to_excel('amazon_review_embedded.xlsx')

In [90]:
df = pd.read_csv('amazon_review_embedded.csv')

In [99]:


df['Embed_sentence'] = df['Embed_sentence'].apply(np.array)
# df['Embed_sentence'] = df['Embed_sentence'].apply(eval)

In [104]:
df 

Unnamed: 0            Review Model Retailer              Comp Model  \
0               0     Canon Pixma TS6420a   Amazon                     NaN   
1               1  HP OfficeJet Pro 9015e   Amazon  Epson - Workforce 4820   
2               2      Canon PIXMA MG3620   Amazon                     NaN   
3               3         Epson - ET-3830   Amazon                     NaN   
4               4           HP ENVY 6055e   Amazon     Canon Pixma TS6420a   
...           ...                     ...      ...                     ...   
10204       10204         Epson - XP-6100   Amazon                     NaN   
10205       10205  Epson - Workforce 4820   Amazon                     NaN   
10206       10206  Epson - Workforce 3820   Amazon                     NaN   
10207       10207         Epson - ET-2800   Amazon                     NaN   
10208       10208      Canon PIXMA TS3520   Amazon                     NaN   

      Review date          Review name  Review rating  \
0      2023-08-10      Ernest Birkholz              5   
1      2022-06-04               mattey              3   
2      2023-03-15              Maria D              4   
3      2022-11-17               Ryan H              5   
4      2022-04-09                  Sam              3   
...           ...                  ...            ...   
10204  2020-10-17     MD ZAKIR HOSSAIN              5   
10205  2022-10-13           mario aste              5   
10206  2023-07-26                  111              1   
10207  2022-07-12  Margarita Rodriguez              1   
10208  2021-09-22        Paul Bridgman              4   

                                                          Review title  \
0                                                          Works great   
1                                              spunky mid size printer   
2                                                         𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜   
3                                   Shaq knows what he's talking about   
4                                                    Not User Friendly   
...                                                                ...   
10204                                                   Quick Delivery   
10205                                                  Awesome printer   
10206                                                      Not working   
10207                                           El artículo llegó roto   
10208  The new printer worked like the one we used to have, very good.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Review Content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [100]:
labels = [
    'Setup', 
    'Connectivity', 
    'Customer Support', 
    'Print Quality', 
    'Print Speed', 
    'Ink supply and Cartridge', 
    'Printer Hardware Robustness and sturdiness', 
    'Control Panel', 
    'Ease of Use', 
    'Firmware', 
    'Business Services and Subscription', 
     'Paper jam',
     'Control Panel', 
    'Other'
]

df_labels = pd.DataFrame({'Labels': labels})
df_labels

,Labels
0,Setup
1,Connectivity
2,Customer Support
3,Print Quality
4,Print Speed
5,Ink supply and Cartridge
6,Printer Hardware Robustness and sturdiness
7,Control Panel
8,Ease of Use
9,Firmware


In [104]:
# label_token = [label.split() for label in labels]

In [ ]:
# def docs2vecs(docs,dictionary):
#     vec1 = [dictionary.doc2bow(doc) for doc in docs]
#     tfid = gensim.models.TfidfModel(vec1)
#     vec2 = [tfidf[vec] for vec in vec1]
#     return vec2

# vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 2, stop_words = 'english')
# X = vectorizer.fit_transform(dataset.data)

In [85]:
# model = Word2Vec(sentences = label_token, vector_size=100, window=5, min_count=1, sg=0)

In [112]:
# from gensim.models import Word2Vec
# import pandas as pd

# # List of labels
# labels = [
#     'Setup', 
#     'Connectivity', 
#     'Customer Support', 
#     'Print Quality', 
#     'Print Speed', 
#     'Ink supply and Cartridge', 
#     'Printer Hardware Robustness and sturdiness', 
#     'Control Panel', 
#     'Ease of Use', 
#     'Firmware', 
#     'Business Services and Subscription', 
#     'Paper jam',
#     'Control Panel', 
#     'Other'
# ]

# # Create a DataFrame with labels
# df_labels = pd.DataFrame({'Labels': labels})

# # Split the labels into words (tokens)
# label_tokens = [label.split() for label in labels]

# # Train a Word2Vec model on label tokens
# model = Word2Vec(sentences=label_tokens, vector_size=100, window=5, min_count=1, sg=0)

# # Function to get the embedding of a label
# def label_embedding(label, model):
#     return model.wv[label]  # Use indexing to retrieve the embedding

# # Apply the label_embedding function to embed labels
# df_labels['Label Embeddings'] = df_labels['Labels'].apply(lambda x: label_embedding(x, model))

In [101]:
def embed_label(text,model):
    embeddings = model([text])
    return embeddings.numpy()[0]

In [102]:
text = 'Connectivity'
input_embedding_vector = embed_label(text,model)

df['similarity'] = df['Embed_sentence'].apply(lambda x: cosine_similarity(input_embedding_vector,x))


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U8320')) -> None

In [103]:
pd.set_option('display.max_colwidth', None)
df_new = df.sort_values(by = 'similarity',ascending = False)
columns = ['Brand','Review Model','Review rating','Review Content']
df_select = df_new[columns]
df_final = df_select[df_select['Brand'] == 'HP'].head(10).reset_index(drop = True)
df_final

,Brand,Review Model,Review rating,Review Content
0,HP,HP Smart Tank 6001,1,Connection problems
1,HP,HP ENVY Inspire 7955e,5,So easy to connect
2,HP,HP DeskJet 2755e,5,Connected to devices with no problem.
3,HP,HP Smart Tank 7301,2,wireless will not stay connected
4,HP,HP ENVY 6455e,2,It keeps going offline
5,HP,HP DeskJet 2755e,3,It seems to be a decent little printer/scanner when it is actually connected to your network. But set up is a pain and it keeps loosing connection. It is very difficult to get it to re-connect. Don't waste your time with this printer.
6,HP,HP ENVY 6055e,5,Easy setup - no issues with connecting to my wifi
7,HP,HP DeskJet 4155e,4,its a printer
8,HP,HP OfficeJet Pro 9025e,1,Poor wifi connection
9,HP,HP OfficeJet Pro 9025e,1,Always offline or unavailable.


In [19]:
# def search_reviews(df, product_description, pprint=True):
#     product_embedding = get_embedding(
#         product_description,
#         engine=embedding_model
#     )
#     df[f"similarity_{product_description}"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

#     results = df.sort_values(f"similarity_{product_description}", ascending=False)
                       
#     return results



# summary_df = []

# for label in labels:
#     label_results = search_reviews(df_filter, label, pprint=True)
    
#     # Apply a similarity threshold to filter rows
#     threshold = 0.85
#     filtered_results = label_results[label_results[f"similarity_{label}"] > threshold]
    
#     # Add the label column to the filtered results
#     filtered_results['topic'] = label
    
#     # Append the filtered results to the summary_df
#     summary_df.append(filtered_results)
    
#     time.sleep(5)

# # Concatenate the filtered results into the final summary DataFrame
# summary_df = pd.concat(summary_df)


In [30]:
# columns_to_drop = ['embedding', 'embedding_splitted']

# # Use the drop method to remove the specified columns
# df_phrase_filtered = df_phrase.drop(columns=columns_to_drop)

# Other

In [38]:
df_filter = df_filter.drop_duplicates(subset = ['Review Model', 'key phrase splitted'])

In [43]:
labels = [
    'Setup', 
    'Connectivity', 
    'Customer Support', 
    'Print Quality', 
    'Print Speed', 
    'Ink supply and Cartridge', 
    'Printer Hardware Robustness and sturdiness', 
    'Control Panel', 
    'Ease of Use', 
    'Firmware', 
    'Business Services and Subscription', 
    'HP Smart App',
     'Paper jam',
     'Control Panel', 
    'Other'
]

In [46]:
def find_most_similar_word(review_embedding, word_vectors, word_list):
    similarities = [cosine_similarity(review_embedding, word_vector) for word_vector in word_vectors]
    most_similar_index = similarities.index(max(similarities))
    return word_list[most_similar_index]  # Return the actual word

# Assuming you have a DataFrame 'df' with 'embedding' column containing review embeddings
df_phrase['Topic'] = df_phrase['embedding_splitted'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


C:\Users\YeTi985\AppData\Local\Temp\ipykernel_17244\3029389664.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phrase['Topic'] = df_phrase['embedding_splitted'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


In [47]:
df_phrase['Topic2'] = df_phrase['embedding'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))

C:\Users\YeTi985\AppData\Local\Temp\ipykernel_17244\3720456180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phrase['Topic2'] = df_phrase['embedding'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


In [48]:
df_phrase 

,Unnamed: 0,Review Model,Retailer,Review date,Review rating,Review title,Review content,Review concat text,Tokens,key phrase,embedding,key phrase splitted,embedding_splitted,Topic,Topic2
6388,6388,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Easy setup,"[0.0028877160511910915, 0.015896936878561974, ...",Setup,Ink supply and Cartridge
6389,6389,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",easy scanning,"[-0.04566308483481407, 0.025933345779776573, 0...",Ease of Use,Ink supply and Cartridge
6390,6390,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Proprietary restrictions,"[0.0037913022097200155, -0.012775414623320103,...",Firmware,Ink supply and Cartridge
6391,6391,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Inflexibility with ink brands,"[-0.0056190043687820435, 0.005538931582123041,...",Ink supply and Cartridge,Ink supply and Cartridge
6401,6401,HP OfficeJet Pro 9025e,Amazon,2023-03-03,5,Great Printer,"After years of my old printer, purchased this ...",Retailer: Amazon - Review model: HP OfficeJet ...,84,"[Fast printing, liked two tray feature, easy w...","[-0.03244659677147865, 0.009338131174445152, -...",Fast printing,"[-0.0462610200047493, 0.024372506886720657, -0...",Print Speed,Print Speed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,10711,HP OfficeJet Pro 9025e,Amazon,2021-05-08,5,Great for working from home,"This really is a great printer, and it's so mu...",Retailer: Amazon - Review model: HP OfficeJet ...,268,"[Excellent for home office use, works seamless...","[-0.014914722181856632, 0.0015333282062783837,...",positive response to free ink plan,"[-0.013421832583844662, -0.01247564610093832, ...",Ink supply and Cartridge,Printer Hardware Robustness and sturdiness
10712,10712,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",Easy setup process,"[0.006355913821607828, 0.021701103076338768, 0...",Setup,Ease of Use
10713,10713,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",App-based setup is convenient,"[-0.006079057231545448, 0.010124939493834972, ...",Ease of Use,Ease of Use
10714,10714,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",Supports multiple connections,"[-0.038474854081869125, 0.005594192072749138, ...",Connectivity,Ease of Use


In [49]:
df_phrase.to_excel('Amazon_9025e_topic.xlsx', index = False)